# GRB Model README

These programs shows you how to calcualte the integrated photon and energy fluxes of for each model using two different methods.

1)  The first example in each file was done by taking the integral of each GRB Function in Wolfram Alpha and then using those integrated equations to estimate fluxes.  An array of energies is passed to the integrated functions and the fluxes are estimated for each increment of energy by calculating the flux at Energy[i] and subtracting it from the flux at Energy[i+1]. 

    ** A link to the Wolfram Alpha Integration will be provided for 
    each model in their own Jupyter Notebooks.
    
<br >
<br >

2)  Using scipy.integrate.quad to run quadrature integration on the origianl GRB function.

## Here we will show you an example using the Power-Law Model.

---

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

## First Example:

### Original Function Integrated

In [2]:
def lpow(engs, flux, *params):
    '''
    ** This function is normalized to 100 keV, which is why the 100's are 
       included in this function.  We integrated the following equation:
       f(E) = N * ((E/100.0)**a)
        a:     power-law index
        N:     normalization
        E:     energy to integrate over, 10 to 10,000 keV
               
       
    engs:  array of energies from 10 to 10,000 keV 
           (lower and upper integration limits).
    
    flux:  array of zero's or nan's to prepare for fluxes to be appended to.
           It's faster to append to an already existing array by doing
           flux[i] = 1.234 rather than flux.append(1.234).  The .append 
           syntax eats up computation time.
           
    *params: a list of parameters (floats) in the order they would be in PYXSPEC.
             You should make sure the function reads them in that same order.
             
    '''
    
    plIdx = float(params[0])    # power-law index

    for i in range(len(engs)-1):
        lowIntegral     = (100.**(-plIdx)) * ((engs[i]**(plIdx+1.)) /(plIdx + 1.))
        highIntegral    = (100.**(-plIdx)) * ((engs[i+1]**(plIdx+1.))/(plIdx + 1.))
        val             = (highIntegral - lowIntegral)
        flux[i]         = val

#### The normalization parameter is left out of this function for 1 reason only:
Without it, these functions are set up exactly as the PYXSPEC functions should be to fit the data and estimate parameters.  This function can be used to fit the data.
See Documentation on adding a local model to PyXspec:
https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/extended.html#local-models-in-python


#### Parameters and Constants:

In [3]:
pars     = [-1.492, 4.05928592E-02]  # plIndex, plNormalization

def get_parVals():
    pars     = [-1.492, 4.05928592E-02]  
    return pars

# Constants:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

In [4]:
Num    = 5000
engs   = np.logspace(1, 4, Num)
flux   = np.zeros(Num)

In [5]:
# WILL STORE CALCULATIONS IN ARRAY 'flux'
lpow(engs, flux, *pars)

#### Photon Flux:
    Photon Flux =  f(E) dE 
    (units: photons s^-1 cm^-2)

In [6]:
# MULTIPLY BY NORMALIZATION PARAMETER
norm     = pars[-1]
flux_ph  = np.sum(flux) * norm

#### Energy Flux:

    Energy Flux =  E * f(E) dE 
    (units: ergs s^-1 cm^-2)
    ** units come out in keV and need to be converted to ergs.

In [7]:
flux_en = np.sum(flux * engs * keVtoerg) * norm

In [8]:
print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))


Photon Flux:  24.758399542 	 photons s^-1 cm^-2
Energy Flux:  1.287664172e-05 	 ergs s^-1 cm^-2



1.  In order to get the function we have in the lpow function above, we first integrate the origianl Power-Law Function in Wolfram Alpha.

2.  We then pass it an array of energies and take the difference between energy[i+1] and energy[i].

3.  Multiply the fluxes by the normalization parameter.  Since it's a constant during the integration, it can be pulled out front and multiplied to the final sum (np.sum(flux) \* norm) or each individual element np.sum(flux \* norm), it does not make a difference.

### Extra Notes:

Make a flux array of 5000 elements that will be filled with flux increments.
An energy array with 5000 energies ranging from 10 to 10,000 keV in log space will be passed to the integrated BAND function and will calculate an increment of flux for each energy element passed to it.  
Those increments will be stored in the flux array of 5000 elements.

After running band(engs, flux, *pars) the fluxes (un-normalized) will be appended to the flux array.  Then you can multiply each flux element by the normalization factor.  
You could pass the normalization to the band function and multiply it within the function instead of at the end.  
I do not do that here because the equation (shown below) was the exact one  used during PYXSPEC fitting.  The only difference between this function and the PYXSPEC one is that the order the variables passed to the band function in PYXSPEC are:  band(engs, params, flux)


If you sum up the 5000 increments in the flux array and then multiply by the the normalization, you are getting the integrated flux from 10 to 10,000 keV.
You are essentially summing up the flux area under the flux curve.


In this example, we leave the BAND normalization parameter out of the function and multiply it at the end.  We do this because PYXSPEC uses this exact function for band (as below) during the Maximum Likelihood fitting process of parameter estimation.  

This is how PYXSPEC estiamtes its flux calculations when you use the commands:

    AllModels.setEnergies("10. 10000.")
    AllModels.calcFlux("10. 10000.0 err")
 
This is a crude method to estimate the flux and is not the  most accurate way.  The best way to calculate the integrated flux is to take the original function and numerically integrate it with scipy.integrate.quad for quadrature integration.

    Photon Flux Units:  photons s^-1 cm^-2
    Energy Flux Units:  ergs s^-1 cm^-2




---

## Second Example:

### Original Function.
This uses scipy.integrate.quad

This way is more precise.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html

In [9]:
def pl(energy):
    '''
    Integrating this function will provide the Integrated Photon Flux.
    
    Can not pass parameters to this function.  Need to call them from
    within.  The reason for this is because scipy.integrate.quad setup
    needs to call a function that is set up to only pass one parameter 
    to, the parameter to integrate over.  Here, that would be energy.
    
    '''
    plIdx, N    = get_parVals()  #power-law index and  its normalization.
    
    eng    = energy
    return N * ((eng/100.0)**plIdx)

def epl(energy):
    '''
    Integrating this function will provide the Integrated Energy Flux.
    Must Convert from keV to ergs after integration.
    '''
    eng    = energy
    return eng * pl(eng)

In [10]:
Flux_Ph = integrate.quad(pl, emin, emax, limit=100)[0]
Flux_En = integrate.quad(epl, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  24.758399542 	 photons s^-1 cm^-2
Energy Flux:  1.288554146e-05 	 ergs s^-1 cm^-2



### If you don't want to call the parameters from inside the function, you can do this:

In [11]:
def pl(energy, *args):
    '''
    Integrating this function will provide the Integrated Photon Flux.
    
    '''
    plIdx, N = args
    eng = energy
    return N * ((eng/100.0)**plIdx)


def epl(energy, *args):
    '''
    Integrating this function will provide the Integrated Energy Flux.
    Must Convert from keV to ergs after integration.
    '''
    plIdx, N = args
    eng = energy
    return eng * pl(eng, *args)

In [12]:
pars = [-1.492, 4.05928592E-02]

Flux_Ph = integrate.quad(pl, emin, emax, args=tuple(pars), limit=100)[0]
Flux_En = integrate.quad(epl, emin, emax, args=tuple(pars), limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  24.758399542 	 photons s^-1 cm^-2
Energy Flux:  1.288554146e-05 	 ergs s^-1 cm^-2

